In [1]:
import numpy as np 
import pandas as pd 
import pickle 
from os.path import join 
from sklearn.decomposition import PCA 
from sklearn.model_selection import GridSearchCV 

import warnings
warnings.filterwarnings('ignore')

In [2]:


# current directory where .pkl files are present (for features extraction)
DATA_DIR = join("./")

def load_batches_to_dict(amount_to_load=3):
    """Loads batches (.pkl files) from disc and returns one concatenated dict (features in csv).
    amount_to_load specifies the number of batches to load, starting from 1."""
    if amount_to_load < 1 or amount_to_load > 3:
        raise "amount_to_load is not a valid number! Try a number between 1 and 3."

    batches_dict = {}  # Initializing

    # Replicating Load Data logic
    print("Loading batch1 ...")
    # path of batch1.pkl ./batch1.pkl
    path1 = join(DATA_DIR, "batch1.pkl")
    # loading in memory
    batch1 = pickle.load(open(path1, 'rb'))

    #remove batteries that do not reach 80% capacity
    del batch1['b1c8']
    del batch1['b1c10']
    del batch1['b1c12']
    del batch1['b1c13']
    del batch1['b1c22']
    
    # updates/replaces the values of dictionary with the new dictionary)
    batches_dict.update(batch1)

    if amount_to_load > 1:
        print("Loading batch2 ...")
        path2 = join(DATA_DIR, "batch2.pkl")
        batch2 = pickle.load(open(path2, 'rb'))

        # There are four cells from batch1 that carried into batch2, we'll remove the data from batch2
        # and put it with the correct cell from batch1
        batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16']
        batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
        add_len = [662, 981, 1060, 208, 482]

        for i, bk in enumerate(batch1_keys):
            batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
            for j in batch1[bk]['summary'].keys():
                if j == 'cycle':
                    batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
                else:
                    batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
            last_cycle = len(batch1[bk]['cycles'].keys())
            for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
                batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]

        del batch2['b2c7']
        del batch2['b2c8']
        del batch2['b2c9']
        del batch2['b2c15']
        del batch2['b2c16']

        # All keys have to be updated after the reordering.
        batches_dict.update(batch1)
        batches_dict.update(batch2)

    if amount_to_load > 2:
        print("Loading batch3 ...")
        path3 = join(DATA_DIR, "batch3.pkl")
        batch3 = pickle.load(open(path3, 'rb'))

        # remove noisy channels from batch3
        del batch3['b3c37']
        del batch3['b3c2']
        del batch3['b3c23']
        del batch3['b3c32']
        del batch3['b3c38']
        del batch3['b3c39']

        batches_dict.update(batch3)

    print("Done loading batches")
    return batches_dict


def build_feature_df(batch_dict):
    """Returns a pandas DataFrame with all originally used features out of a loaded batch dict"""

    print("Start building features ...")

    from scipy.stats import skew, kurtosis
    from sklearn.linear_model import LinearRegression
    
    # 124 cells (3 batches)
    n_cells = len(batch_dict.keys())

    ## Initializing feature vectors:
    # numpy vector with 124 zeros
    cycle_life = np.zeros(n_cells)
    # 1. delta_Q_100_10(V)
    minimum_dQ_100_10 = np.zeros(n_cells)
    variance_dQ_100_10 = np.zeros(n_cells)
    skewness_dQ_100_10 = np.zeros(n_cells)
    kurtosis_dQ_100_10 = np.zeros(n_cells)

    # 2. Discharge capacity fade curve features
    slope_lin_fit_2_100 = np.zeros(n_cells)  # Slope of the linear fit to the capacity fade curve, cycles 2 to 100
    intercept_lin_fit_2_100 = np.zeros(n_cells)  # Intercept of the linear fit to capavity face curve, cycles 2 to 100
    discharge_capacity_2 = np.zeros(n_cells)  # Discharge capacity, cycle 2
    diff_discharge_capacity_max_2 = np.zeros(n_cells)  # Difference between max discharge capacity and cycle 2

    # 3. Other features
    mean_charge_time_2_6 = np.zeros(n_cells)  # Average charge time, cycle 2 to 6
    minimum_IR_2_100 = np.zeros(n_cells)  # Minimum internal resistance
   
    diff_IR_100_2 = np.zeros(n_cells)  # Internal resistance, difference between cycle 100 and cycle 2

    # Classifier features
    minimum_dQ_5_4 = np.zeros(n_cells)
    variance_dQ_5_4 = np.zeros(n_cells)
    cycle_550_clf = np.zeros(n_cells)
    
    # iterate/loop over all cells.
    for i, cell in enumerate(batch_dict.values()):
        cycle_life[i] = cell['cycle_life'] 
        # 1. delta_Q_100_10(V)
        c10 = cell['cycles']['10']
        c100 = cell['cycles']['100']
        dQ_100_10 = c100['Qdlin'] - c10['Qdlin']

        minimum_dQ_100_10[i] = np.log10(np.abs(np.min(dQ_100_10)))
        variance_dQ_100_10[i] = np.log(np.abs(np.var(dQ_100_10)))
        skewness_dQ_100_10[i] = np.log(np.abs(skew(dQ_100_10)))
        kurtosis_dQ_100_10[i] = np.log(np.abs(kurtosis(dQ_100_10)))

        # 2. Discharge capacity fade curve features
        # Compute linear fit for cycles 2 to 100:
        q = cell['summary']['QD'][1:100].reshape(-1, 1)  # discharge cappacities; q.shape = (99, 1);
        X = cell['summary']['cycle'][1:100].reshape(-1, 1)  # Cylce index from 2 to 100; X.shape = (99, 1)
        linear_regressor_2_100 = LinearRegression()
        linear_regressor_2_100.fit(X, q)

        slope_lin_fit_2_100[i] = linear_regressor_2_100.coef_[0]
        intercept_lin_fit_2_100[i] = linear_regressor_2_100.intercept_
        discharge_capacity_2[i] = q[0][0]
        diff_discharge_capacity_max_2[i] = np.max(q) - q[0][0]

        # 3. Other features
        mean_charge_time_2_6[i] = np.mean(cell['summary']['chargetime'][1:6])
        minimum_IR_2_100[i] = np.min(cell['summary']['IR'][1:100])
        diff_IR_100_2[i] = cell['summary']['IR'][100] - cell['summary']['IR'][1]

        # Classifier features
        c4 = cell['cycles']['4']
        c5 = cell['cycles']['5']
        dQ_5_4 = c5['Qdlin'] - c4['Qdlin']
        minimum_dQ_5_4[i] = np.log10(np.abs(np.min(dQ_5_4)))
        variance_dQ_5_4[i] = np.log10(np.var(dQ_5_4))
        cycle_550_clf[i] = cell['cycle_life'] >= 550

    # combining all featues in one big matrix where rows are the cells and colums are the features
    # note last two variables below are labels/targets for ML i.e cycle life and cycle_550_clf
    features_df = pd.DataFrame({
        "cell_key": np.array(list(batch_dict.keys())),
        "minimum_dQ_100_10": minimum_dQ_100_10,
        "variance_dQ_100_10": variance_dQ_100_10,
        "skewness_dQ_100_10": skewness_dQ_100_10,
        "kurtosis_dQ_100_10": kurtosis_dQ_100_10,
        "slope_lin_fit_2_100": slope_lin_fit_2_100,
        "intercept_lin_fit_2_100": intercept_lin_fit_2_100,
        "discharge_capacity_2": discharge_capacity_2,
        "diff_discharge_capacity_max_2": diff_discharge_capacity_max_2,
        "mean_charge_time_2_6": mean_charge_time_2_6,
        "minimum_IR_2_100": minimum_IR_2_100,
        "diff_IR_100_2": diff_IR_100_2,
        "minimum_dQ_5_4": minimum_dQ_5_4,
        "variance_dQ_5_4": variance_dQ_5_4,
        "cycle_life": cycle_life,
        "cycle_550_clf": cycle_550_clf
    })

    print("Done building features")
    return features_df




In [3]:
# calling function to load from disk
all_batches_dict = load_batches_to_dict()
# function to build features for ML
features_df = build_feature_df(all_batches_dict)

# save all features 124 in disk in rebuild_features.csv file
save_csv_path = join(DATA_DIR, "rebuild_features.csv")
features_df.to_csv(save_csv_path, index=False)
print("Saved features to ", save_csv_path)

Loading batch1 ...
Loading batch2 ...
Loading batch3 ...
Done loading batches
Start building features ...
Done building features
Saved features to  ./rebuild_features.csv


In [4]:
def train_val_split(features, regression_type="full", model="regression"):
    # only three versions are allowed.
    assert regression_type in ["full", "variance", "discharge"]
    
    # dictionary to hold the features indices for each model version.
    features = {
        "full": [1, 2, 5, 6, 7, 9, 10, 11],
        "variance": [2],
        "discharge": [1, 2, 3, 4, 7, 8]
    }
    # get the features for the model version (full, variance, discharge)
    feature_indices = features[regression_type]
    # get all cells with the specified features 
    model_features = features_df.iloc[:,feature_indices]
    # get last two columns (cycle life and classification)
    labels = features_df.iloc[:, -2:] 
    # labels are (cycle life ) for regression other wise (0/1) for classsification
    labels = labels.iloc[:, 0] if model == "regression" else labels.iloc[:, 1]
    
    # split data in to train/primary_test/and secondary test
    train_cells = np.arange(1,83,2) 
    val_cells = np.arange(0,84,2)
    test_cells = np.arange(84, 124, 1)
    
    # get cells and their features of each set and convert to numpy for further computations
    x_train = np.array(model_features.iloc[train_cells])
    x_val = np.array(model_features.iloc[val_cells])
    x_test = np.array(model_features.iloc[test_cells])
    
    # target values or labels for training
    y_train = np.array(labels.iloc[train_cells])
    y_val = np.array(labels.iloc[val_cells])
    y_test = np.array(labels.iloc[test_cells])
    
    # return 3 sets
    return {"train": (x_train, y_train), "val": (x_val, y_val), "test": (x_test, y_test)}

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn import linear_model
 
def regression(datasets, pca=False, normalize=True, log_target=True,
               n_components=5, alpha=1e-4, l1_ratio=0.99, model="elastic"):
    # get three sets
    x_train, y_train = datasets.get("train")
    x_val, y_val = datasets.get("val")
    x_test, y_test = datasets.get("test")
    
    
    if pca:
        # create pca object with n_components (n_components is the reduced feature dimension)
        _pca = PCA(n_components=n_components)
        _pca.fit(x_train)
        # transfrom data to reduced features
        x_train = _pca.transform(x_train)
        x_val = _pca.transform(x_val)
        x_test = _pca.transform(x_test)
    if normalize:
        # create normalization object to normalize data
        scaler = StandardScaler()
        # find normalization params from train set, (mean, std)
        scaler.fit(x_train)
        # normalize sets (0 mean, 1 std)
        x_train = scaler.transform(x_train)
        x_val = scaler.transform(x_val)
        x_test = scaler.transform(x_test) 


    
    if model == "elastic":
        print ("Elastic net")
        regr = ElasticNet(random_state=4, alpha=alpha, l1_ratio=l1_ratio)
    else:
        print ("Lasso net")
        regr = Lasso(alpha=alpha)
    # labels/ targets might be converted to log version based on choice
    targets = np.log(y_train) if log_target else y_train
    # fit regression model
    regr.fit(x_train, targets)

    # predict values/cycle life for all three sets
    pred_train = regr.predict(x_train)
    pred_val = regr.predict(x_val)
    pred_test = regr.predict(x_test)
    
    if log_target:
        # scale up the preedictions
        pred_train, pred_val, pred_test = np.exp(pred_train), np.exp(pred_val), np.exp(pred_test)
    
    # mean percentage error (same as paper)
    error_train = mean_absolute_percentage_error(y_train, pred_train)*100
    error_val = mean_absolute_percentage_error(y_val, pred_val)*100
    error_test = mean_absolute_percentage_error(y_test, pred_test)*100
    
    print(f"Regression Error batch 3 (test (secondary)):  {error_test}%" )
    print(f"Regression Error (Train):, {error_train}%")
    print(f"Regression Error (validation (primary) test): {error_val}%")
    

In [34]:
if __name__ == "__main__":
    # skip this cell if features are already build
    # read features from CSV file
    import pandas as pd
    features_df = pd.read_csv("./rebuild_features.csv")
    print ("Full")
    features = train_val_split(features_df, regression_type="full")
    regression(features, pca=False, normalize=False, n_components=3, alpha=0.0005,
               l1_ratio=1, log_target=False, model="elastic")

    print ("\nDischarge")
    features = train_val_split(features_df, regression_type="discharge")
    regression(features, pca=False, normalize=False, n_components=3, alpha=0.0001,
               l1_ratio=1, log_target=True, model="elastic")

    print ("\nVariance")
    features = train_val_split(features_df, regression_type="variance")
    regression(features, pca=False, normalize=False, n_components=3, alpha=0.001,
               l1_ratio=1, log_target=True, model="elastic")


Full
Elastic net
Regression Error batch 3 (test (secondary)):  16.48701158264002%
Regression Error (Train):, 11.014193993605865%
Regression Error (validation (primary) test): 22.87890922366772%

Discharge
Elastic net
Regression Error batch 3 (test (secondary)):  16.647625827135297%
Regression Error (Train):, 11.461057782933471%
Regression Error (validation (primary) test): 18.663078336586363%

Variance
Elastic net
Regression Error batch 3 (test (secondary)):  11.689134915392648%
Regression Error (Train):, 16.069316810065658%
Regression Error (validation (primary) test): 16.928284800729614%


In [35]:
def classification(datasets, pca=False, normalize=True,
               n_components=5, C=1, tol=0.01):
    x_train, y_train = datasets.get("train")
    x_val, y_val = datasets.get("val")
    x_test, y_test = datasets.get("test")
    
    
    if pca:
        _pca = PCA(n_components=n_components)
        _pca.fit(x_train)
        x_train = _pca.transform(x_train)
        x_val = _pca.transform(x_val)
        x_test = _pca.transform(x_test)
    if normalize:
        scaler = StandardScaler()
        scaler.fit(x_train)
        x_train = scaler.transform(x_train)
        x_val = scaler.transform(x_val)
        x_test = scaler.transform(x_test)  
    
    
    clf = LogisticRegression(penalty='l2',C=C, tol=tol, max_iter=1000)
    clf.fit(x_train, y_train)

    # Accuracy calculation
    acc_train = clf.score(x_train, y_train)*100
    acc_val = clf.score(x_val, y_val)*100
    acc_test = clf.score(x_test, y_test)*100
    
    print(f"Accuracy batch 3 (test (secondary)):  {acc_test}%" )
    print(f"Accuracy (Train):, {acc_train}%")
    print(f"Accuracy (validation (primary) test): {acc_val}%")
    

In [36]:
from sklearn.linear_model import LogisticRegression
print ("Full")
features_cls = train_val_split(features_df, regression_type="full", model="classification")
classification(features_cls, C=9, tol=0.01)

print ("\nVariance")
features_cls = train_val_split(features_df, regression_type="variance", model="classification")
classification(features_cls, C=0.02, tol=0.001)

print ("\nDischarge")
features_cls = train_val_split(features_df, regression_type="discharge", model="classification")
classification(features_cls, C=1, tol=0.1)

Full
Accuracy batch 3 (test (secondary)):  92.5%
Accuracy (Train):, 97.5609756097561%
Accuracy (validation (primary) test): 90.47619047619048%

Variance
Accuracy batch 3 (test (secondary)):  95.0%
Accuracy (Train):, 78.04878048780488%
Accuracy (validation (primary) test): 71.42857142857143%

Discharge
Accuracy batch 3 (test (secondary)):  97.5%
Accuracy (Train):, 92.6829268292683%
Accuracy (validation (primary) test): 88.09523809523809%
